In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/vishveshdumbre/nvidia-180-days-minute-dataset-2025")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: IanD24
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/vishveshdumbre/nvidia-180-days-minute-dataset-2025


100%|██████████| 1.55M/1.55M [00:00<00:00, 684MB/s]

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available()else 'cpu'

In [3]:
# Import Stocks datasets from Kaggle
# Clearning and Normalizing data by deviding by largest values in columns

full_df = pd.read_csv("/content/nvidia-180-days-minute-dataset-2025/nvda_180_days_minute_data.csv")
train_df = full_df.iloc[: int(len(full_df) * 0.8)]
test_df = full_df.iloc[int(len(full_df) * 0.8) :]

train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df.drop(columns=['symbol', 'timestamp'], inplace=True)
test_df.drop(columns=['symbol', 'timestamp'], inplace=True)

train_df = train_df / train_df.max()
test_df = test_df / test_df.max()

# Creating Labels
train_labels = train_df["open"].shift(-1) / train_df["open"]
test_labels = test_df["open"].shift(-10) / test_df["open"]


# Getting rid of last rows of each dataset
train_labels = train_labels[:-1]
test_labels = test_labels[:-1]

train_df = train_df[:-1]
test_df = test_df[:-1]


# Turning it all into arrays
train_df = np.array(train_df)
test_df = np.array(test_df)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

/tmp/ipython-input-3943682080.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(inplace=True)
/tmp/ipython-input-3943682080.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.dropna(inplace=True)
/tmp/ipython-input-3943682080.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns=['symbol', 'timestamp'], inplace=True)
/tmp/ipython-input-3943682080.py:12: SettingWithCopyWarning: 
A value is

In [4]:
# Creating Dataset class to allow np arrarys to be converted to tensors for use with pytorch module
class squential_dataset(Dataset):
  def __init__(self, X, Y, sequence_length):
    self.sequence_length = sequence_length
    self.X = []
    self.Y = []

    for i in range(len(X) - sequence_length):
      self.X.append(X[i : i+sequence_length])
      self.Y.append(Y[i : i+sequence_length])

    self.X = torch.tensor(self.X, dtype = torch.float).to(device)
    self.Y = torch.tensor(self.Y, dtype = torch.float).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.Y[index]

In [5]:
# Assigning data to variables for training, validation, and testing data

sequence_length = 10

training_data = squential_dataset(train_df[ : int(0.8 * len(train_df)), :], train_labels[ :int(0.8 * len(train_labels))], sequence_length)
validation_data = squential_dataset(train_df[ int(0.8 * len(train_df)) : , :], train_labels[int(0.8 * len(train_labels)) : ], sequence_length)
testing_data = squential_dataset(test_df, test_labels, sequence_length)

/tmp/ipython-input-266683371.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  self.X = torch.tensor(self.X, dtype = torch.float).to(device)


In [6]:
# Creating dataloader iterables for batching data in training
train_dataloader = DataLoader(training_data, batch_size = 16)
validation_dataloader = DataLoader(validation_data, batch_size = 16)
testing_dataloader = DataLoader(testing_data, batch_size = 16)

In [7]:
# Setting hyper paramaters for the network
hidden_size = 5
num_classes = 1
num_epochs = 3
batch_size = 100
learning_rate = 0.001

input_size = 5
num_layers = 2

In [8]:
# Buildingl class for my neural network including the types of layers and activation functions
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_stacked_layers):
    super().__init__()

    self.hidden_size = hidden_size
    self.num_stacked_layers = num_stacked_layers
    self.LSTM = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    self.linear_layer = nn.Linear(hidden_size, 1)


  def forward(self, x):
    h0 = torch.zeros(self.num_stacked_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_stacked_layers, x.size(0), self.hidden_size).to(device)

    out, _ = self.LSTM(x, (h0, c0))
    out = out[:, -1, :]
    out = self.linear_layer(out)

    return out

model = LSTM(input_size, hidden_size, num_layers).to(device)

In [9]:
def train_one_epoch():
  model.train(True)
  running_loss = 0.0

  for batch in train_dataloader:
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)
    loss = loss_function(output, y_batch.unsqueeze(1))
    running_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

def validate_epoch():
  model.train(False)
  running_loss = 0.0

  for batch in validation_dataloader:
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    with torch.no_grad():
      output = model(x_batch)
      loss = loss_function(output, y_batch.unsqueeze(1))
      running_loss += loss.item()

  avg_loss = running_loss / len(validation_dataloader)

  print(f"Loss: {avg_loss} ")
  print("------------------------")


In [10]:
optimizer = Adam(model.parameters(), lr = 1e-3)
epochs = 10
loss_function = nn.MSELoss()

for epoch in range(epochs):
  train_one_epoch()
  validate_epoch()




/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([16, 1, 10])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([10, 1, 10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([12, 1, 10])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  ret

Loss: 3.4877810092753195e-06 
------------------------
Loss: 3.6674996213817114e-06 
------------------------
Loss: 5.325723083167884e-06 
------------------------
Loss: 4.044053473724792e-06 
------------------------
Loss: 2.2991685167703422e-06 
------------------------
Loss: 1.8408724098263452e-06 
------------------------
Loss: 1.759299688837886e-06 
------------------------
Loss: 1.6816684533793591e-06 
------------------------
Loss: 1.6156381910782514e-06 
------------------------
Loss: 1.5618656700559324e-06 
------------------------
